### Watson creds & Model parameters

In [1]:
!pip install ibm-watson
!pip install pymilvus
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.4/359.4 kB 9.1 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.8 MB/s eta 0:00:00
  Created wheel for ibm-watson: filename=ibm_watson-10.0.0-py3-none-any.whl size=361969 sha256=d11f6bd387d805b2749a84a150a9ee62871ba37ca06a8fcc5c1b085bfe64544b
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/f3/66/bb/8f7ce4c04a9b1d165c0dcccb0dbb1494af51a2f77f16336251
Successfully built ibm-watson
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 88.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.7/515.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.3 MB/s e

In [ ]:
import os
from ibm_watsonx_ai import APIClient, Credentials

credentials = Credentials(
    url="https://eu-gb.ml.cloud.ibm.com",
    api_key= input("Enter your IBM CLoud API Key:")
)

In [3]:
llm_model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"

In [4]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

In [5]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")

In [6]:
from ibm_watsonx_ai.foundation_models import ModelInference

llm_model = ModelInference(
	model_id = llm_model_id,
	params = parameters,
	credentials = credentials,
	project_id = project_id,
	space_id = space_id
	)

### Init Milvus Client

In [ ]:
from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType
import time 

MILVUS_HOST = input("Enter your Milvus Host (e.g., in03-xxxx.serverless.aws-...): ")
MILVUS_TOKEN = input("Enter your Milvus API Key: ")

#Initialize MilvusClient
try:
    client = MilvusClient(
        uri=f"https://{MILVUS_HOST}", # Use HTTPS for Zilliz Cloud
        token=MILVUS_TOKEN
    )
    print(" Successfully initialized MilvusClient.")
    print(f"Connected to Milvus at {MILVUS_HOST}")

except Exception as e:
    print(f" Error connecting to Milvus: {e}")
    # If connection fails, there's no point in proceeding
    exit() # This will stop execution of the cell

✅ Successfully initialized MilvusClient.
Connected to Milvus at in03-9c74f36fb672dbb.serverless.aws-eu-central-1.cloud.zilliz.com


In [8]:
#Define Collection Schema

from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType

#Define your desired collection name
collection_name = "usecase_collection" 

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Milvus will auto-generate IDs
    FieldSchema(name="article_text", dtype=DataType.VARCHAR, max_length=2500,), # Your text content field
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384), # Embedding vector field
]
schema = CollectionSchema(fields, "Collection for RAG demo with article text chunks")

#Create Collection
if client.has_collection(collection_name=collection_name):
    print(f"\nCollection '{collection_name}' already exists.")

else:
    try:
        print(f"\nCreating collection '{collection_name}'...")
        client.create_collection(
            collection_name=collection_name,
            schema=schema
        )
        print(f" Collection '{collection_name}' created successfully.")
    except Exception as e:
        print(f" Error creating collection '{collection_name}': {e}")
        exit() # Stop if collection creation fails


Collection 'usecase_collection' already exists.


In [9]:
index_params = MilvusClient.prepare_index_params()

#Add an index on the vector field.
index_params.add_index(
    field_name="vector",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

try:
    print(f"\nCreating index on '{collection_name}'...")
    client.create_index(
        collection_name=collection_name,
        index_params=index_params
    )
    print(f" Index created successfully for collection '{collection_name}'.")
except Exception as e:
    print(f" Error creating index for collection '{collection_name}': {e}")


Creating index on 'usecase_collection'...
 Index created successfully for collection 'usecase_collection'.


In [11]:
#pip install trafilatura

In [ ]:
import trafilatura

url = input("Enter the Wikipedia URL:")

downloaded = trafilatura.fetch_url(url)

if downloaded:
    extracted_text = trafilatura.extract(downloaded, include_comments=False, include_tables=False)
    #print(extracted_text)
else:
    print("Failed to fetch the content.")


## Prepare chunks for vectorization

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50,
    length_function = len,
    add_start_index = True,
)

chunks = text_splitter.create_documents([extracted_text])
print(f"Original text split into {len(chunks)} chunks.")


#print (chunks)

Original text split into 121 chunks.


In [12]:
from sentence_transformers import SentenceTransformer

try:
    model_path = "ibm-granite/granite-embedding-107m-multilingual" # 384 dim model
    print("SentenceTransformer model loaded successfully.")
except Exception as e:
    print(f"Error loading SentenceTransformer model: {e}")
    exit()

#Vectorize and Insert Chunks using MilvusClient (List of Dictionaries Format) ---
print(f"\nVectorizing and inserting {len(chunks)} chunks into '{collection_name}'...")
inserted_count = 0

# Prepare a list to hold all records for a potential batch insert (more efficient)
records_to_insert = []

for i, chunk_doc in enumerate(chunks):
    chunk_text = chunk_doc.page_content

    # IMPORTANT: Check if the chunk_text length exceeds your schema's max_length
    if len(chunk_text) > 2500:
        print(f"Warning: Chunk {i+1} has length {len(chunk_text)}, which exceeds schema max_length (2500). "
              "This chunk might cause an error during insertion or be truncated by Milvus."
              "Consider adjusting chunk_size in RecursiveCharacterTextSplitter or schema max_length.")
        # You might want to skip this chunk or truncate it explicitly here if it's consistently too long.

    # Generate embeddings for the current chunk
    passage_embeddings = model.encode(chunk_text).tolist()

 
    record = {
        "article_text": chunk_text,     # This is a string (the chunk content)
        "vector": passage_embeddings    # This is a list of floats (the vector)
    }
    records_to_insert.append(record)

    

# --- Perform the actual insertion after preparing all records ---
if records_to_insert:
    try:
        # Pass the list of dictionaries directly to client.insert()
        out = client.insert(
            collection_name=collection_name,
            data=records_to_insert # This is the list of dictionaries
        )
        inserted_count = out.get('insert_count', 0) # Get actual count from response
        print(f"\n Successfully inserted {inserted_count} chunks into '{collection_name}'.")
        # print(f"Milvus auto-generated IDs for this batch: {out.get('ids', 'N/A')}")
    except Exception as e:
        print(f" Error during batch insertion: {e}")
else:
    print("No chunks to insert.")

client.flush(collection_name=collection_name) # Ensure all inserted data is persisted
print("Data flushed to Milvus.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer model loaded successfully.

Vectorizing and inserting 121 chunks into 'usecase_collection'...

 Successfully inserted 121 chunks into 'usecase_collection'.
Data flushed to Milvus.


In [13]:
from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType

# --- 2. Define the new collection name for relations ---
relations_collection_name = "relations_collection" 

# --- 3. Define Relation Collection Schema ---
relation_fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Milvus will auto-generate IDs
    FieldSchema(name="relation_phrase", dtype=DataType.VARCHAR, max_length=512), # Combined string for vectorization
    FieldSchema(name="relation_type", dtype=DataType.VARCHAR, max_length=128),  # e.g., 'managerOf', 'employedBy'
    FieldSchema(name="arg1_text", dtype=DataType.VARCHAR, max_length=256),    # Text of the first argument
    FieldSchema(name="arg2_text", dtype=DataType.VARCHAR, max_length=256),    # Text of the second argument
    FieldSchema(name="original_sentence", dtype=DataType.VARCHAR, max_length=1024), # Original sentence context
    FieldSchema(name="relation_vector", dtype=DataType.FLOAT_VECTOR, dim=384) # Embedding vector for the relation_phrase
]
relation_schema = CollectionSchema(relation_fields, "Collection for extracted entity relations and their contexts")

# --- 4. Create Relations Collection (if it doesn't exist) ---
if client.has_collection(collection_name=relations_collection_name):
    print(f"\nCollection '{relations_collection_name}' already exists.")

else:
    try:
        print(f"\nCreating collection '{relations_collection_name}'...")
        client.create_collection(
            collection_name=relations_collection_name,
            schema=relation_schema
        )
        print(f"✅ Collection '{relations_collection_name}' created successfully.")
    except Exception as e:
        print(f"❌ Error creating collection '{relations_collection_name}': {e}")
        exit() # Stop if collection creation fails




Collection 'relations_collection' already exists.


In [14]:
from pymilvus import MilvusClient, DataType 

# --- 5. Define and Create Index on the new Relations Collection ---

relation_index_params = MilvusClient.prepare_index_params()

relation_index_params.add_index(
    field_name="relation_vector",        # Name of the vector field to index
    index_type="IVF_FLAT",               # Index algorithm (e.g., IVF_FLAT, HNSW)
    metric_type="COSINE",                # Similarity metric (e.g., COSINE, L2)
    params={"nlist": 128}                # Parameters for the chosen index type
)

try:
    print(f"\nCreating index on '{relations_collection_name}'...")
    client.create_index(
        collection_name=relations_collection_name,
        index_params=relation_index_params
    )
    print(f"✅ Index created successfully for collection '{relations_collection_name}'.")

except Exception as e:
    # If the error is "Index already exists", it's fine. Other errors need attention.
    if "Index already exist" in str(e):
        print(f" Index on '{relations_collection_name}' for field '{relation_index_params.field_name}' already exists. Skipping creation.")
    else:
        print(f" Error creating index for collection '{relations_collection_name}': {e}")
        # Consider whether to exit or continue based on the severity of the error

print(f"\nMilvus setup for '{relations_collection_name}' complete.")
#-----------------------------------------------------------------------------


Creating index on 'relations_collection'...
✅ Index created successfully for collection 'relations_collection'.

Milvus setup for 'relations_collection' complete.


In [ ]:
from sentence_transformers import SentenceTransformer  # Ensure this is imported if not already
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, RelationsOptions

# --- Configuration ---
API_KEY = input("Enter your IBM Watson NLU API Key: ")
SERVICE_URL = input("Enter your IBM Watson NLU Service URL: ")

# The text you want to analyze for relations
text_to_analyze = extracted_text

# --- Initialize and Analyze ---
try:
    authenticator = IAMAuthenticator(API_KEY)
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2022-08-10',
        authenticator=authenticator
    )
    natural_language_understanding.set_service_url(SERVICE_URL)

    print("Watson Natural Language Understanding service initialized successfully.\n")

    features = Features(
        relations=RelationsOptions()
    )

    print("Analyzing text for relations...\n")
    response = natural_language_understanding.analyze(
        text=text_to_analyze,
        features=features
    ).get_result()

    #print (response)

    # Optional: print or process the response
    print("Analysis successful. Relations extracted.\n")
    relations_to_insert = response.get("relations", [])

except Exception as e:
    print(f"An error occurred during NLU initialization or analysis: {str(e)}")
    relations_to_insert = []

Watson Natural Language Understanding service initialized successfully.

Analyzing text for relations...

Analysis successful. Relations extracted.



In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('ibm-granite/granite-embedding-107m-multilingual')

def process_relation(relation):
    relation_type = relation.get('type')
    sentence = relation.get('sentence')
    arguments = relation.get('arguments', [])
    

    if len(arguments) >= 2 and all('text' in arg for arg in arguments[:2]):
        arg1_text = arguments[0]['text']
        arg2_text = arguments[1]['text']
        combined_phrase = f"{arg1_text} {relation_type} {arg2_text}"
        #print (combined_phrase)

        relation_embedding = model.encode(combined_phrase).tolist()

        return {
            "relation_phrase": combined_phrase,
            "relation_type": relation_type,
            "arg1_text": arg1_text,
            "arg2_text": arg2_text,
            "original_sentence": sentence, # retrieve original sentence
            "relation_vector": relation_embedding # search only by relation
        }
    else:
        print(f"Skipping relation due to insufficient arguments: {relation}")
        return None

relations_to_insert = []

if 'relations' in response and response['relations']:
    print("\nProcessing and vectorizing extracted relations for insertion...")
    for relation in response['relations']:
        record = process_relation(relation)
        if record:
            relations_to_insert.append(record)
else:
    print("No relations found in the provided text to process.")



Processing and vectorizing extracted relations for insertion...


In [17]:
# --- Insert Relations into Milvus (Batch Insertion) ---
if relations_to_insert:
    try:
        print(f"\nInserting {len(relations_to_insert)} relations into '{relations_collection_name}'...")
        insert_response = client.insert(
            collection_name=relations_collection_name,
            data=relations_to_insert # This is the list of dictionaries, one for each relation record
        )
        print(f"✅ Successfully inserted {insert_response.get('insert_count', 0)} relations.")
        client.flush(collection_name=relations_collection_name) # Ensure data persistence
        print("Relations data flushed to Milvus.")
    except Exception as e:
        print(f"❌ Error inserting relations into Milvus: {e}")
else:
    print("No valid relations were prepared for insertion.")


Inserting 870 relations into 'relations_collection'...
✅ Successfully inserted 870 relations.
Relations data flushed to Milvus.


In [ ]:
from sentence_transformers import SentenceTransformer
from pymilvus import MilvusClient


model = SentenceTransformer("ibm-granite/granite-embedding-107m-multilingual")

question = input("Enter your question here rekated to the article:")
question_vector = model.encode(question).tolist()

relation_results = client.search(
    collection_name="relations_collection",
    data=[question_vector],
    limit=50,
    output_fields=["relation_phrase", "arg1_text", "arg2_text", "relation_type", "original_sentence"]
)
chunk_results = client.search(
    collection_name="usecase_collection",
    data=[question_vector],
    limit=10,
    output_fields=["article_text", "id"]
)

# print (relation_results)
# print (chunk_results)

top_relations = []
top_chunks = []

print(" Top Relations:")
for  rel in relation_results[0]:
    #print(f"- {rel['entity']['relation_phrase']} (score: {rel['distance']:.4f})")
    top_relations.append(rel)  # Append inside the loop


print("\n Contextual Chunks:")
for chuk in chunk_results[0]:
    #print(f"- {chuk['entity']['article_text']} (score: {chuk['distance']:.4f})")
    top_chunks.append(chuk)  # Append inside the loop



 Top Relations:

 Contextual Chunks:


In [19]:
def generate_answer_with_context(relations, chunks, question):
    """
    Sends a prompt to the model with high-score relations, contextual chunks, and a user question.
    Guides the model to generate a final answer based on that context only.
    """
 
    # Format the relations and context for display
    #relations_text = "\n".join([f"- {res['entity']['relation_phrase']} (score: {res['distance']:.4f})" for res in relations[0]])
    #chunks_text = "\n".join([f"- {res['entity']['original_sentence']} (score: {res['distance']:.4f})" for res in chunks[0]])

    # Final system-guided prompt
    system_prompt = (
        "You are a helpful and knowledgeable assistant.\n"
        "Below are:\n"
        "- Extracted factual relations (ranked by relevance score).\n"
        "- Supporting contextual information from text.\n"
        "- A user question.\n\n"
        "Your task is to analyze the high-score relations and context, and generate an accurate, clear, and concise final answer "
        "based *only* on the provided information. Do not make assumptions or include external knowledge.\n"
        "If the answer is not clearly found in the relations or context, state: 'The provided context does not contain enough information to answer this question.'\n\n"
        f"--- Relations ---\n{relations}\n\n"
        f"--- Context ---\n{chunks}\n\n"
        f"--- Question ---\n{question}\n\n"
        f"Final Answer:"
    )

    return system_prompt


# 2. Generate the final prompt
final_prompt = generate_answer_with_context(top_relations, top_chunks, question)

#print(final_prompt)

# Final Prompt & Answer

In [20]:
chat_messages = [];
chat_messages.append({
    "role": f"system",
    "content": final_prompt
})
chat_messages.append({"role": "user", "content": question})
generated_response = llm_model.chat(messages=chat_messages)
print(generated_response['choices'][0]['message']['content'])

Rania is the mother of Princess Iman. According to the context, "King Abdullah and Queen Rania have four children: Crown Prince Hussein, Princess Iman, Princess Salma, and Prince Hashem." This indicates that Rania is the mother of Princess Iman. 

Final Answer: Mother and daughter.
